In [ ]:
#===========================================================================================
# ALBERT Futher-PreTraining 예시
# => MLM(Masked Language Model)만으로 Further Pre-Train 시키는 예시
# => AlbertForMaskedLM 이용
#===========================================================================================

import torch
from transformers import AlbertTokenizer, AlbertForMaskedLM, AlbertModel
from tqdm.notebook import tqdm
from transformers import AdamW, get_linear_schedule_with_warmup
from os import sys
sys.path.append('..')
from myutils import GPU_info, seed_everything, mlogging, AccuracyForMaskedToken, SaveBERTModel

device = GPU_info()
print(device)

#seed 설정
seed_everything(333)

#logging 설정
logger =  mlogging(loggername="Albertfpt", logfilename="../../log/Albertfpt")


In [ ]:
# 신규 모델에 파일을 불러옴


vocab_path = '../../data11/model/albert/albert-base-v2-ftp-4' # 앞에서 만들어진 spiece_new.model 이 있는 경로 지정해줌
model_path = '../../data11/model/albert/albert-base-v2-ftp-4'

tokenizer = AlbertTokenizer.from_pretrained(vocab_path)
print(len(tokenizer))

model = AlbertForMaskedLM.from_pretrained(model_path)

# resize_token_embeddings 으로 신규 tokenizer 사이즈로 지정 해줌.
model.resize_token_embeddings(len(tokenizer))
model.to(device)

In [ ]:
model.num_parameters()

In [ ]:
# text tokenizer 해봄.
text = "모코엠시스에서는 문서중앙화 및 보안파일서버 솔루션인 엠파워를 출시하였다."
token_ids = tokenizer.encode_plus(text, max_length=128, padding="max_length", return_tensors="pt")
print(token_ids)

In [ ]:
# 각 스페셜 tokenid를 구함
CLStokenid = tokenizer.convert_tokens_to_ids('[CLS]')
SEPtokenid = tokenizer.convert_tokens_to_ids('[SEP]')
UNKtokenid = tokenizer.convert_tokens_to_ids('<UNK>')
PADtokenid = tokenizer.convert_tokens_to_ids('<pad>')
MASKtokenid = tokenizer.convert_tokens_to_ids('[MASK]')

print(CLStokenid)
print(SEPtokenid)
print(UNKtokenid)
print(PADtokenid)
print(MASKtokenid)


In [ ]:
input_corpus = '../../data11/korpora/kowiki_20190620/wiki_20190620_mecab_false_0311.txt'
eval_corpus = '../../data11/korpora/kowiki_20190620/wiki_eval_test.txt'
token_max_len = 128
batch_size = 32
#==============================================================================
# DataLoader 설정으로 메모리 속도 계선방법
#
# 1) num_workers>0 설정
# num_workers=0훈련 또는 이전 프로세스가 완료된 후에만 데이터 로드를 실행하는 반면,
# num_workers > 0 값으로 설정하면 특히 I/O 및 대용량 데이터 증가에 대한 프로세스가 가속된다
# => 따라서 Dataloader(dataset, num_workers=4*num_GPU) 식으로 설정함
#
# 2) pin_memory=True 설정
# GPU는 CPU의 페이징 가능한 메모리에서 직접 데이터에 액세스할 수 없습니다. 
# 이 설정 pin_memory=True은 CPU 호스트의 데이터에 대한 스테이징 메모리를 직접 할당하고 
# 페이징 가능 메모리에서 스테이징 메모리(즉, 고정된 메모리, 페이지 잠금 메모리)로 데이터를 
# 전송하는 시간을 절약할 수 있다.
# => Dataloader(dataset, pin_memory=True)
#=========================================================================


from torch.utils.data import DataLoader, RandomSampler
import sys
sys.path.append('..')
from myutils import MLMDataset

# 각 스페셜 tokenid를 구함
CLStokenid = tokenizer.convert_tokens_to_ids('[CLS]')
SEPtokenid = tokenizer.convert_tokens_to_ids('[SEP]')
UNKtokenid = tokenizer.convert_tokens_to_ids('<UNK>')
PADtokenid = tokenizer.convert_tokens_to_ids('<pad>')
MASKtokenid = tokenizer.convert_tokens_to_ids('[MASK]')
print('CLSid:{}, SEPid:{}, UNKid:{}, PADid:{}, MASKid:{}'.format(CLStokenid, SEPtokenid, UNKtokenid, PADtokenid, MASKtokenid))

# mask 처리할 목록
Maskvocab_list = [
    '▁정보', '▁데이터', '▁서비스', '▁인터페이스', '▁환경', '▁통합', '▁이름', '▁저장', '▁성능', '▁연결', '▁테이블', 
    '▁웹', '▁Application', '▁소프트웨어', '▁관리자', '▁제품', '▁기간', '▁모듈', '▁구조', '▁네트워크', '▁호스트', 
    '▁정책', '▁software', '▁클라우드', '▁노드', '▁result', '▁명령', '▁시험', '▁업데이트', '▁Added', '▁로그인', 
    '▁클라이언트', '▁편집', '▁mark', '▁전파','▁다이얼로그', '▁디스플레이', '▁사무실', '▁오퍼레이션', '▁로그아웃', '▁유닉스', 
    '▁Soft', '▁관측', '▁윈도우즈', '▁시그널', '▁프린팅', '▁모바일게임', '▁생성자', '▁webserver', '▁카카오톡', '▁Keyboard'
]

train_dataset = MLMDataset(corpus_path = input_corpus,
                           tokenizer = tokenizer, 
                           CLStokeinid = CLStokenid ,   # [CLS] 토큰 id
                           SEPtokenid = SEPtokenid ,    # [SEP] 토큰 id
                           UNKtokenid = UNKtokenid ,    # [UNK] 토큰 id
                           PADtokenid = PADtokenid,    # [PAD] 토큰 id
                           Masktokenid = MASKtokenid,   # [MASK] 토큰 id
                           max_sequence_len=token_max_len,  # max_sequence_len)
                           mlm_probability=0.15,
                           overwrite_cache=True,
                           Maskvocab_list = Maskvocab_list
                          )


# 학습 dataloader 생성
# => tenosor로 만듬
train_loader = DataLoader(train_dataset, 
                          batch_size=batch_size, 
                          #shuffle=True, # dataset을 섞음
                          sampler=RandomSampler(train_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                          num_workers=4,   # *num_workers = 4*num_GPU 로 설정(프로세스가 빨라짐)
                          pin_memory=True  # *pin_memory=True은 페이징 가능 메모리에서 스테이징 메모리로 전송하는 시간을 줄여줌
                         )

#===============================================================================
# eval dataloader 생성
eval_dataset = MLMDataset(corpus_path = eval_corpus,
                          tokenizer = tokenizer, 
                          CLStokeinid = CLStokenid ,   # [CLS] 토큰 id
                          SEPtokenid = SEPtokenid ,    # [SEP] 토큰 id
                          UNKtokenid = UNKtokenid ,    # [UNK] 토큰 id
                          PADtokenid = PADtokenid,    # [PAD] 토큰 id
                          Masktokenid = MASKtokenid,   # [MASK] 토큰 id
                          max_sequence_len=token_max_len,  # max_sequence_len)
                          mlm_probability=0.15,
                          overwrite_cache=True
                          )


# eval dataloader 생성
# => tenosor로 만듬
eval_loader = DataLoader(eval_dataset, 
                         batch_size=batch_size, 
                         #shuffle=True, # dataset을 섞음
                         sampler=RandomSampler(eval_dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                         num_workers=4,   # *num_workers = 4*num_GPU 로 설정(프로세스가 빨라짐)
                         pin_memory=True  # *pin_memory=True은 페이징 가능 메모리에서 스테이징 메모리로 전송하는 시간을 줄여줌
                         )
#===============================================================================

print(train_dataset[0])

In [ ]:
##################################################
epochs = 20            # epochs
learning_rate = 3e-5  # 학습률
OUTPATH = '../../data11/model/albert/albert-base-v2-ftp-4'
##################################################

# optimizer 적용
optimizer = AdamW(model.parameters(), 
                 lr=learning_rate, 
                 eps=1e-8) # 0으로 나누는 것을 방지하기 위한 epsilon 값(10^-6 ~ 10^-8 사이 이값 입력합)

# 총 훈련과정에서 반복할 스탭
total_steps = len(train_loader)*epochs
warmup_steps = total_steps * 0.1 #10% of train data for warm-up

# 손실률 보여줄 step 수
p_itr = int(len(train_loader)*0.2)  
    
# step마다 모델 저장
save_steps = int(total_steps * 0.3)
    
# 스캐줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=warmup_steps, 
                                            num_training_steps=total_steps)

itr = 1
total_loss = 0
total_len = 0
total_correct = 0
total_test_correct = 0
total_test_len = 0
    
list_train_loss = []
list_train_acc = []
list_validation_acc = []

model.to(device)

model.zero_grad(set_to_none=True)# 그래디언트 초기화(*set_to_none=True 로 설정하면, 그래디언트 업데이트시, 쓰기작업만 수행되어 속도가 빨라진다)
for epoch in tqdm(range(epochs)):

    model.train() # 훈련모드로 변환
    for data in tqdm(train_loader):
        #optimizer.zero_grad()
        model.zero_grad(set_to_none=True)# 그래디언트 초기화(*set_to_none=True 로 설정하면, 그래디언트 업데이트시, 쓰기작업만 수행되어 속도가 빨라진다)
        
        # 입력 값 설정
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        token_type_ids = data['token_type_ids'].to(device)       
        labels = data['labels'].to(device)
        #print('Labels:{}'.format(labels))
        
            
        # 모델 실행
        outputs = model(input_ids=input_ids, 
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        labels=labels)
        
       
        # 출력값 loss,logits를 outputs에서 얻어옴
        loss = outputs.loss
        logits = outputs.logits
        #print('Loss:{}, logits:{}'.format(loss, logits))
       
        # optimizer 과 scheduler 업데이트 시킴
        loss.backward()   # backward 구함
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)   # 그래디언트 클리핑 (gradient vanishing이나 gradient exploding 방지하기 위한 기법)
        optimizer.step()  # 가중치 파라미터 업데이트(optimizer 이동)
        scheduler.step()  # 학습률 감소
        
        # ***further pretrain 에는 손실률 계산을 넣지 않음
        # 정확도 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
        
        # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
        # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
        with torch.no_grad():
            
            # 손실률 계산
            total_loss += loss.item()
            
            #===========================================
            # 정확도(Accurarcy) 계산
            correct, masked_len = AccuracyForMaskedToken(logits, labels, input_ids, MASKtokenid)           
            total_correct += correct.sum().item() 
            total_len += masked_len 
            #=========================================
                
            # 주기마다 test(validataion) 데이터로 평가하여 손실류 계산함.
            if itr % p_itr == 0:
                
                train_loss = total_loss/p_itr
                train_acc = total_correct/total_len
                       
                ####################################################################
                # 주기마다 eval(validataion) 데이터로 평가하여 손실류 계산함.
                # 평가 시작
                model.eval()

                #for data in tqdm(eval_loader):
                for data in eval_loader:
                    # 입력 값 설정
                    input_ids = data['input_ids'].to(device)
                    attention_mask = data['attention_mask'].to(device)
                    token_type_ids = data['token_type_ids'].to(device)       
                    labels = data['labels'].to(device)

                    with torch.no_grad():
                        # 모델 실행
                        outputs = model(input_ids=input_ids, 
                                       attention_mask=attention_mask,
                                       token_type_ids=token_type_ids,
                                       labels=labels)

                        # 출력값 loss,logits를 outputs에서 얻어옴
                        #loss = outputs.loss
                        logits = outputs.logits

                        #===========================================
                        # 정확도(Accurarcy) 계산
                        correct, masked_len = AccuracyForMaskedToken(logits, labels, input_ids, MASKtokenid)           
                        total_test_correct += correct.sum().item() 
                        total_test_len += masked_len 
                        #=========================================

                val_acc = total_test_correct/total_test_len
                    
                logger.info('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Train Acc: {:.4f}, Val Acc:{}({}/{})'.format(epoch+1, epochs, itr, train_loss, train_acc, val_acc, total_test_correct, total_test_len))
                    
                list_train_loss.append(train_loss)
                list_train_acc.append(train_acc)
                list_validation_acc.append(val_acc)
                 
                # 변수들 초기화    
                total_loss = 0
                total_len = 0
                total_correct = 0
                total_test_correct = 0
                total_test_len = 0
                ####################################################################

            if itr % save_steps == 0:
                #전체모델 저장
                SaveBERTModel(model, tokenizer, OUTPATH, epochs, learning_rate, batch_size)

        itr+=1
   

In [ ]:
# 모델 저장
SaveBERTModel(model, tokenizer, OUTPATH, epochs, learning_rate, batch_size)

In [ ]:
# 그래프로 loss 표기
#!pip install matplotlib
import matplotlib.pyplot as plt

plt.plot(list_train_loss, label='Train Loss')
#plt.plot(list_train_acc, label='Train Accuracy')
#plt.plot(list_validation_acc, label='Eval Accuracy')
plt.legend()
plt.show()

plt.plot(list_train_acc, label='Train Accuracy')
plt.plot(list_validation_acc, label='Eval Accuracy')
plt.legend()
plt.show()